### Import Libraries

In [26]:
import numpy as np
import pandas as pd
import re
import os
import pickle
from sklearn.preprocessing import LabelEncoder
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import Data

In [27]:
movie_path = '../assets/final_movie.pkl'
rating_path = '../assets/final_rating.pkl'
model_path = '../assets/model.pkl'
user_item_path = '../assets/user_movie_interaction.pkl'
dataset_path = '../assets/dataset.pkl'
user_feats_path = '../assets/user_feats.pkl'
csr_path = '../assets/csr.pkl'

In [28]:
movie = pd.read_pickle(movie_path)
new_rating = pd.read_pickle(rating_path)

In [29]:
with open(model_path, 'rb') as m:
    model = pickle.load(m)

In [30]:
with open(user_item_path, 'rb') as u:
    user_item_interaction = pickle.load(u)

In [31]:
with open(dataset_path, 'rb') as d:
    dataset = pickle.load(d)

In [32]:
with open(user_feats_path, 'rb') as u:
    user_feats = pickle.load(u)

In [33]:
with open(csr_path, 'rb') as c:
    user_item_csr = pickle.load(c)

### Helpers

In [34]:
def recommend_top5(model, movie, dataset, user_id=None, new_user_feature=None, k=5):
    
    nmovie=movie.set_index('movieId')
    
    max_user_id = max(new_rating.userId)
    if user_id is None:
        user_id = max_user_id +1
    if user_id > max_user_id:
        for genre in unique_genre:
            if genre not in list(new_user_feature.keys()):
                new_user_feature[genre] = 0
        dataset.fit_partial(users=[user_id],user_features=unique_genre)
        new_user_feature = [user_id,new_user_feature]
        new_user_feature = dataset.build_user_features([new_user_feature],normalize=False)

    user_id_map = dataset.mapping()[0][user_id] 
    scores = model.predict(user_id_map, np.arange(n_items),user_features=new_user_feature)
    rank = np.argsort(-scores)
    selected_movie_id =np.array(list(dataset.mapping()[2].keys()))[rank]
    top_items = nmovie.loc[selected_movie_id]

        
    return top_items['clean_movie_title'][:5]

In [35]:
def create_random_user_info(user_id = None):
    
    """This function will create random user information regarding their genre and interests.
    Input: 
    user_id (None): If None, then we assume this is a new user.
                    If we specify the user information, then it will overwrite their previous information.
    unique_genre (list): contains unique genre in the dataset. 
                           
    Output:
    user_info_dict (dict): It will contain user information regarding the genres and their interests.
    """
    
    # create an empty dictionary to save the results
    user_info_dict = {}
    
    for genre in unique_genre:
        rand_score = round(np.random.rand(1)[0], 4)
        user_info_dict[genre] = rand_score
        
    return user_info_dict

### Test

In [36]:
n_users, n_items = user_item_csr.shape

In [37]:
unique_genre = sorted(set(map(lambda genre: \
                                  genre.strip(),','.join(map(lambda genre: str(genre), movie['new_genres'].values))\
                                  .split(','))))

### Testing

change a bit so that we can see what they've watched and their recommended movies

In [38]:
recommend_top5(model,
            movie,
            dataset,
            user_id = 2)

movieId
480                                  Jurassic Park
316                                       Stargate
1210    Star Wars: Episode VI - Return of the Jedi
260             Star Wars: Episode IV - A New Hope
736                                        Twister
Name: clean_movie_title, dtype: object

In [39]:
recommend_top5(model,
            movie,
            dataset,
            user_id = None,
            new_user_feature= {'Comedy':1, 'Mystery': 0.5, 'Adventure':0.8})

movieId
344              Ace Ventura: Pet Detective
1073    Willy Wonka & the Chocolate Factory
2804                     Christmas Story, A
588                                 Aladdin
223                                  Clerks
Name: clean_movie_title, dtype: object

In [40]:
recommend_top5(model,
            movie,
            dataset,
            user_id = None,
            new_user_feature= {'Comedy':0.6, 'Mystery': 0.5, 'Adventure':0.8})

movieId
344     Ace Ventura: Pet Detective
223                         Clerks
588                        Aladdin
597                   Pretty Woman
1265                 Groundhog Day
Name: clean_movie_title, dtype: object

- random testing

In [41]:
rand_user_info_1 = create_random_user_info()
rand_user_info_2 = create_random_user_info()
rand_user_info_3 = create_random_user_info()

In [42]:
recommend_top5(model,
            movie,
            dataset,
            user_id = None,
            new_user_feature= rand_user_info_1)

movieId
5618      Spirited Away (Sen to Chihiro no kamikakushi)
5971              My Neighbor Totoro (Tonari no Totoro)
78499                                       Toy Story 3
109487                                     Interstellar
134853                                       Inside Out
Name: clean_movie_title, dtype: object

In [43]:
recommend_top5(model,
            movie,
            dataset,
            user_id = None,
            new_user_feature= rand_user_info_2)

movieId
183869        Hereditary
194200         Hell Fest
169498     Buffalo Dance
158743    Occupation 101
160420            Thithi
Name: clean_movie_title, dtype: object

In [44]:
recommend_top5(model,
            movie,
            dataset,
            user_id = None,
            new_user_feature= rand_user_info_3)

movieId
109487                                     Interstellar
79132                                         Inception
5618      Spirited Away (Sen to Chihiro no kamikakushi)
78499                                       Toy Story 3
58559                                  Dark Knight, The
Name: clean_movie_title, dtype: object